# Academic QA System with GraphRAG

- Author: [Yongdam Kim](https://github.com/dancing-with-coffee)
- Design: 
- Peer Review: 
- This is a part of [LangChain Open Tutorial](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial)

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/99-TEMPLATE/00-BASE-TEMPLATE-EXAMPLE.ipynb) [![Open in GitHub](https://img.shields.io/badge/Open%20in%20GitHub-181717?style=flat-square&logo=github&logoColor=white)](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/99-TEMPLATE/00-BASE-TEMPLATE-EXAMPLE.ipynb)

## Overview

이 튜토리얼은 GraphRAG를 이용하여 논문 내용 잘 참고하여 더 잘 대답하는 QA 시스템에 대한 구현을 다룹니다. 

GraphRAG는 Microsoft에서 만든 Graph를 이용하여 text의 local & global 정보를 잘 추출하여 답변하는 새로운 시스템입니다.

하지만, Microsoft에서 발표한 GraphRAG 구현체는 Langchain과 결합되어 있지 않아 사용에 어려움이 있습니다.

이러한 점을 개선한 langchain-graphrag 라이브러리를 사용하여 Langchain에서 GraphRAG를 구현할 수 있습니다.

이번 튜토리얼에서는 langchain-graphrag를 사용하여 최신 AI 관련 논문들에 대해 상세하게 답변하는 QA system을 만드는 것에 대해 배웁니다.

![GraphRAG](./assets/08-academicqasystem-graphrag-pipeline-.png)

[논문: From Local to Global-A Graph RAG Approach to Query-Focused Summarization](https://arxiv.org/abs/2404.16130) 

### Table of Contents


### References

----

## Environment Setup

Set up the environment. You may refer to [Environment Setup](https://wikidocs.net/257836) for more details.

**[Note]**
- `langchain-opentutorial` is a package that provides a set of easy-to-use environment setup, useful functions and utilities for tutorials. 
- You can checkout the [`langchain-opentutorial`](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) for more details.

In [1]:
%%capture --no-stderr
!pip install langchain-opentutorial

In [6]:
# Install required packages
from langchain_opentutorial import package

package.install(
    [
        "langsmith",
        "langchain",
        "langchain_core",
        "langchain_community",
        "langchain-graphrag",
        "jq",
    ],
    verbose=False,
    upgrade=False,
)

In [3]:
# Set environment variables
from langchain_opentutorial import set_env

set_env(
    {
        "OPENAI_API_KEY": "",
        "LANGCHAIN_API_KEY": "",
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "08-AcademicQASystem",  # title 과 동일하게 설정해 주세요
    }
)

Environment variables have been set successfully.


You can alternatively set API keys such as `OPENAI_API_KEY` in a `.env` file and load them.

[Note] This is not necessary if you've already set the required API keys in previous steps.

In [ ]:
# Load API keys from .env file
from dotenv import load_dotenv

load_dotenv(override=True)

## Load arXiv PDFs

이번 튜토리얼에서는 arXiv 데이터셋을 사용합니다. arXiv는 최신 논문이 올라와 있는 web archive이며, 모든 논문들이 PDF 형태로 올라와 있습니다. 공식적으로 제공하는 모든 pdf 파일이 올라와있는 공식 github repo가 있으나, 전체 데이터가 1TB 정도가 되고 AWS에서만 전체 PDF를 다운로드 받을 수 있게 되어있어 이 튜토리얼에서는 직접 선정한 몇몇 PDF에 대해서만 진행합니다.

- 전체 데이터셋 링크 : https://github.com/mattbierbaum/arxiv-public-datasets

In [13]:
# GraphRAG paper 불러오기
from langchain.document_loaders import PyPDFLoader

# PyPDFLoader는 한 페이지 단위씩 불러옵니다.
loader = PyPDFLoader("./data/2404.16130v1.pdf")
docs = loader.load()
print(f"Loaded {len(docs)} documents.")
print(docs[0].page_content)

Loaded 15 documents.
From Local to Global: A Graph RAG Approach to
Query-Focused Summarization
Darren Edge1†Ha Trinh1†Newman Cheng2Joshua Bradley2Alex Chao3
Apurva Mody3Steven Truitt2
Jonathan Larson1
1Microsoft Research
2Microsoft Strategic Missions and Technologies
3Microsoft Office of the CTO
{daedge,trinhha,newmancheng,joshbradley,achao,moapurva,steventruitt,jolarso }
@microsoft.com
†These authors contributed equally to this work
Abstract
The use of retrieval-augmented generation (RAG) to retrieve relevant informa-
tion from an external knowledge source enables large language models (LLMs)
to answer questions over private and/or previously unseen document collections.
However, RAG fails on global questions directed at an entire text corpus, such
as “What are the main themes in the dataset?”, since this is inherently a query-
focused summarization (QFS) task, rather than an explicit retrieval task. Prior
QFS methods, meanwhile, fail to scale to the quantities of text indexed by typi

## Text Chunking and Text Extracting

이번 단계에서는 **쿼리 라우팅** 과 **문서 평가** 를 수행합니다. 이 과정은 **Adaptive RAG** 의 중요한 부분으로, 효율적인 정보 검색과 생성에 기여합니다.

- **쿼리 라우팅** : 사용자의 쿼리를 분석하여 적절한 정보 소스로 라우팅합니다. 이를 통해 쿼리의 목적에 맞는 최적의 검색 경로를 설정할 수 있습니다.
- **문서 평가** : 검색된 문서의 품질과 관련성을 평가하여 최종 결과의 정확성을 높입니다. 

이 단계는 **Adaptive RAG** 의 핵심 기능을 지원하며, 정확하고 신뢰할 수 있는 정보 제공을 목표로 합니다.

In [16]:
from langchain_core.documents import Document
from langchain_graphrag.indexing import TextUnitExtractor
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=64)
text_unit_extractor = TextUnitExtractor(text_splitter=splitter)

df_text_units = text_unit_extractor.run(docs)
df_text_units

Processing documents ...: 100%|██████████| 15/15 [00:00<00:00, 478.48it/s]


,document_id,id,text_unit
0,fbdf5c8e-794a-4034-abd5-0f6541a88412,daed0fe6-3d46-4e1a-88cd-ef885016b8ea,From Local to Global: A Graph RAG Approach to\...
1,fbdf5c8e-794a-4034-abd5-0f6541a88412,dccb316b-15d9-4bfe-924a-eec2b1619a6f,tion from an external knowledge source enables...
2,fbdf5c8e-794a-4034-abd5-0f6541a88412,0a6a68d8-ee83-41aa-b52b-dece6f04633c,RAG systems. To combine the strengths of these...
3,fbdf5c8e-794a-4034-abd5-0f6541a88412,ee359a20-5e11-4ff4-b2f9-0d54e15e2c9f,"question, each community summary is used to ge..."
4,fbdf5c8e-794a-4034-abd5-0f6541a88412,d8e81165-54af-47ba-98c2-1dfe09fa725e,approaches is forthcoming at https://aka .ms/g...
...,...,...,...
117,52176a94-547a-4f07-a1f5-2bdc7984b294,09c07a52-3bb6-480f-80c1-2c6f733f1b78,with chain-of-thought reasoning for knowledge-...
118,52176a94-547a-4f07-a1f5-2bdc7984b294,4e6fa665-2bab-41c2-9502-1f44005db346,"Wang, Y ., Lipka, N., Rossi, R. A., Siu, A., Z..."
119,52176a94-547a-4f07-a1f5-2bdc7984b294,1a39e102-4931-43a3-aac4-19f556480b1b,Empirical Methods in Natural Language Processi...
120,cfbf1189-ebf6-4f30-8b6f-d19a0dd387a0,2dd4d3c2-5bc6-41af-83b6-34a67a53a310,"Yao, L., Peng, J., Mao, C., and Luo, Y . (2023..."


### Entity Relationship Extraction

GraphRAG는 chunking으로 나눈 텍스트에서 Entity, Relationship을 추출하여 Knowledge Graph를 자동으로 만들어줍니다.

Knowledge Graph를 만들 때는 LLM을 사용하며, 이번 튜토리얼에서는 속도 및 비용 이슈를 고려하여 gpt-4o-mini를 사용합니다. LLM은 사전에 정의된 prompt를 이용하여 entity와 relationship을 추출하는데 사용됩니다.

In [ ]:
# 대략 12분 정도 소요
from langchain_graphrag.indexing.graph_generation import EntityRelationshipExtractor
from langchain_openai import ChatOpenAI

er_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

# There is a static method provide to build the default extractor
extractor = EntityRelationshipExtractor.build_default(llm=er_llm)
text_unit_graphs = extractor.invoke(df_text_units)

In [18]:
for index, g in enumerate(text_unit_graphs):
    print("---------------------------------")
    print(f"Graph: {index}")
    print(f"Number of nodes - {len(g.nodes)}")
    print(f"Number of edges - {len(g.edges)}")
    print(g.nodes())
    print(g.edges())
    print("---------------------------------")

---------------------------------
Graph: 0
Number of nodes - 11
Number of edges - 8
['DARREN EDGE', 'HA TRINH', 'NEWMAN CHENG', 'JOSHUA BRADLEY', 'ALEX CHAO', 'APURVA MODY', 'STEVEN TRUITT', 'JONATHAN LARSON', 'MICROSOFT RESEARCH', 'MICROSOFT STRATEGIC MISSIONS AND TECHNOLOGIES', 'MICROSOFT OFFICE OF THE CTO']
[('DARREN EDGE', 'MICROSOFT RESEARCH'), ('HA TRINH', 'MICROSOFT RESEARCH'), ('NEWMAN CHENG', 'MICROSOFT STRATEGIC MISSIONS AND TECHNOLOGIES'), ('JOSHUA BRADLEY', 'MICROSOFT STRATEGIC MISSIONS AND TECHNOLOGIES'), ('ALEX CHAO', 'MICROSOFT OFFICE OF THE CTO'), ('APURVA MODY', 'MICROSOFT OFFICE OF THE CTO'), ('STEVEN TRUITT', 'MICROSOFT STRATEGIC MISSIONS AND TECHNOLOGIES'), ('JONATHAN LARSON', 'MICROSOFT RESEARCH')]
---------------------------------
---------------------------------
Graph: 1
Number of nodes - 3
Number of edges - 2
['RAG', 'LLMS', 'QFS']
[('RAG', 'LLMS'), ('RAG', 'QFS')]
---------------------------------
---------------------------------
Graph: 2
Number of nodes - 3


In [24]:
# 추출한 모든 entity(node)중에 하나를 검색
text_unit_graphs[0].nodes["MICROSOFT RESEARCH"]

{'type': 'ORGANIZATION',
 'description': ['Microsoft Research is a division of Microsoft focused on advanced research and development, and is involved in the paper on Query-Focused Summarization.'],
 'text_unit_ids': ['daed0fe6-3d46-4e1a-88cd-ef885016b8ea']}

In [25]:
# 추출한 entity간의 edge(relationship)을 확인
text_unit_graphs[0].edges[("MICROSOFT RESEARCH", "DARREN EDGE")]

{'weight': 1.0,
 'description': ['Darren Edge is a researcher at Microsoft Research, contributing to the paper on Query-Focused Summarization'],
 'text_unit_ids': ['daed0fe6-3d46-4e1a-88cd-ef885016b8ea']}

### Graph Generation

GraphRAG에서는 추출한 모든 entity와 relationship을 사용하지 않고, 합쳐질 수 있는 많은 정보들을 합쳐서 사용합니다. 이 과정을 Summarization이라고 부릅니다.

GraphRAG는 element summarization을 통해서 search의 기능을 향상시킬 수 있으며, 전체 맥락을 잘 이해하는 Global Search의 성능을 높이는데 큰 기여를 합니다.

In [27]:
# 18분 소요
from langchain_graphrag.indexing.graph_generation import (
    GraphsMerger,
    EntityRelationshipDescriptionSummarizer,
    GraphGenerator,
)

graphs_merger = GraphsMerger()

es_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

summarizer = EntityRelationshipDescriptionSummarizer.build_default(llm=es_llm)

graph_generator = GraphGenerator(
    er_extractor=extractor,
    graphs_merger=GraphsMerger(),
    er_description_summarizer=summarizer,
)

graph = graph_generator.run(df_text_units)

Summarizing relationship descriptions: 100%|██████████| 694/694 [00:21<00:00, 31.64it/s]


AttributeError: 'tuple' object has no attribute 'nodes'

In [36]:
print(f"Number of nodes - {len(graph[0].nodes)}")
print(f"Number of edges - {len(graph[0].edges)}")

Number of nodes - 478
Number of edges - 694
Number of nodes - 478
Number of edges - 694


## Local Search through Knowledge Graph

GraphRAG를 통해 만들어진 Knowledge Graph를 이용해서 Local Search와 Global Search를 수행합니다. Local Search는 특정 단락이나 내용에서 내용을 찾을 때 유용하고, Global Search는 전체적인 맥락에서 답변을 얻기 좋은 방법입니다.

In [ ]:
# from langchain_chroma.vectorstores import Chroma as ChromaVectorStore
# from langchain_graphrag.query.global_search import GlobalSearch
# from langchain_graphrag.query.local_search import (
#     LocalSearch,
#     LocalSearchPromptBuilder,
#     LocalSearchRetriever,
# )
# from langchain_graphrag.query.local_search.context_builders import (
#     ContextBuilder,
# )
# from langchain_graphrag.query.local_search.context_selectors import (
#     ContextSelector,
# )
# from langchain_graphrag.types.graphs.community import CommunityLevel
# from langchain_graphrag.utils import TiktokenCounter

# # Reload the vector Store that stores
# # the entity name & description embeddings
# entities_vector_store = ChromaVectorStore(
#     collection_name="entity_name_description",
#     persist_directory=str(vector_store_dir),
#     embedding_function=OpenAIEmbeddings(model="text-embedding-3-small"),
# )

# # Build the Context Selector using the default
# # components; You can supply the various components
# # and achieve as much extensibility as you want
# # Below builds the one using default components.
# context_selector = ContextSelector.build_default(
#     entities_vector_store=entities_vector_store,
#     entities_top_k=10,
#     community_level=cast(CommunityLevel, level),
# )

# # Context Builder is responsible for taking the
# # result of Context Selector & building the
# # actual context to be inserted into the prompt
# # Keeping these two separate further increases
# # extensibility & maintainability
# context_builder = ContextBuilder.build_default(
#     token_counter=TiktokenCounter(),
# )

# # load the artifacts
# artifacts = load_artifacts(artifacts_dir)

# # Make a langchain retriever that relies on
# # context selection & builder
# retriever = LocalSearchRetriever(
#     context_selector=context_selector,
#     context_builder=context_builder,
#     artifacts=artifacts,
# )

# # Build the LocalSearch object
# local_search = LocalSearch(
#     prompt_builder=LocalSearchPromptBuilder(),
#     llm=er_llm,
#     retriever=retriever,
# )

# # it's a callable that returns the chain
# search_chain = local_search()

# # you could invoke
# # print(search_chain.invoke(query))

# # or, you could stream
# for chunk in search_chain.stream(query):
#     print(chunk, end="", flush=True)